In [10]:
import random
def calculate_score(hand):
    score = 0
    ace_count = 0
    for card in hand:
        rank = card
        if rank in ['J', 'Q', 'K']:
            score += 10
        elif rank == 'A':
            score += 11
            ace_count += 1
        else:
            score += int(rank)
    
    # Ace 조정 (11 또는 1로 계산)
    while score > 21 and ace_count > 0:
        score -= 10
        ace_count -= 1
    
    return score
class BlackJackGameSystem:
  def __init__(self,game_num, log_mode):
      self.deck=self.create_deck()
      self.bet_m=0
      self.player_money=500
      self.card_logs={"player":[], "bot":[], "dealer":[]}
      self.log_mode=log_mode
      if self.log_mode:
        print_game(game_num)
      
  def create_deck(self,suffle_num=1):
      ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'] * 4
      deck = [rank for rank in ranks]
      for i in range(suffle_num):
        random.shuffle(deck)
      return deck
  def init_game(self):
    self.deck=self.create_deck()
    self.bet_m=0
    self.card_logs={"player":[], "bot":[], "dealer":[]}
    
  def is_player_alive(self):
    return self.player_money>0
  
  def player_bust_check(self):
    player_hand=self.card_logs['player']
    player_score= calculate_score(player_hand)
    if player_score>21:
      # print(player_score,"이고",player_hand,"로 player는 bust입니다")
      return True
    else:
      # print(player_score,"로 bust가 아닙니다.")
      return False
  


  def give_card(self, role='bot'):
    card=self.deck.pop()
    if role =='dealer':
      self.card_logs['dealer']+=[card]
    elif role=='player':      
      self.card_logs['player']+=[card]
    elif role=='bot':      
      self.card_logs['bot']+=[card]
    
    return [card]

  def show_first_turn_cards(self):
    card_logs4show=dict()
    card_logs4show['player']=self.card_logs['player'].copy()
    card_logs4show['bot']=self.card_logs['bot'].copy()
    card_logs4show['dealer']=self.card_logs['dealer'].copy()
    card_logs4show['dealer'][1]='hide'
    
    return card_logs4show
  
  def get_money_from_player(self, player, bet_m):
    self.bet_m+=bet_m
    player.money -= self.bet_m
    self.player_money -=self.bet_m
    self.integrity_money_test(player)
    
  def settle_the_money(self, player, player_score, dealer_score):
    
    player_win=False
    show_logs=dict()
    show_logs['player']= self.card_logs['player']
    show_logs['dealer']= self.card_logs['dealer']

    if player_score>21:
      if self.log_mode:
        print("플레이어 버스트 딜러 승리",end=" ")
    elif dealer_score > 21:
      player_win=True
      if self.log_mode:
        print("딜러 버스트 플레이어 승리.",end=" ")
    elif player_score >= dealer_score:
      player_win=True
      if self.log_mode:
        print("플레이어 승리",end=" ")
    elif player_score < dealer_score:
      if self.log_mode:
        print("딜러 승리",end=" ")
        
    if player_win:
        player.money += self.bet_m *2
        self.player_money+=self.bet_m *2
        self.bet_m=0
        self.integrity_money_test(player)        
    if self.log_mode:
      print(show_logs,end=" ")
      print("플레이어 남은돈",self.player_money)
  def integrity_money_test(self, player):
    assert player.money==self.player_money, "장부에 기입된 돈이 다릅니다, 사용자가 검은돈을 사용하는 것으로 의심됩니다."

  
  
class AIBot:
  def __init__(self,role='player'):
    self.role=role
    self.hand=[]
    self.money=500
  def clear(self):
    self.hand=[]
  def get_cards(self, card):
    self.hand += card


def print_game(game_num):
  print("==================================================================")
  print(f"========================{game_num}번째 게임=================================")
  print("==================================================================")
      




In [4]:
def play_black_jack(log_mode):
  player=Player('player')
  p2=AIBot('player')
  p3=AIBot('player')
  p4=AIBot('player')
  dealer=AIBot('dealer')
  GOAL_MONEY=800
  GameSystem=BlackJackGameSystem(game_num=game_num,log_mode=log_mode)
  while GameSystem.is_player_alive() and GameSystem.player_money<=GOAL_MONEY:
    player.clear()
    p2.clear()
    p3.clear()
    p4.clear()
    dealer.clear()
    GameSystem.init_game()
    for i in range(2):
        player.get_cards(GameSystem.give_card(role="player"))
        p2.get_cards(GameSystem.give_card(role="bot"))
        p3.get_cards(GameSystem.give_card(role="bot"))
        p4.get_cards(GameSystem.give_card(role="bot"))
        dealer.get_cards(GameSystem.give_card(role="dealer"))
    
    current_state= GameSystem.show_first_turn_cards().copy()
    # print(current_state)
    
    #check bet_money
    bet_money=player.bet(current_state)
    assert bet_money >=20, "최소 배팅금은 20만원 입니다."
    
    GameSystem.get_money_from_player(player, bet_money)
    player_decision= player.draw_decision(current_state)
    
    if player_decision:
      player.get_cards(GameSystem.give_card(role="player"))
    
    while calculate_score(dealer.hand) <=16:
      dealer.get_cards(GameSystem.give_card(role="dealer"))
    
    player_score=calculate_score(player.hand)
    dealer_score=calculate_score(dealer.hand)
    GameSystem.settle_the_money(player, player_score,dealer_score)
    if GameSystem.player_money > GOAL_MONEY:
      return True
    elif GameSystem.player_money < 20:
      return False
    
        

In [23]:
class Player(AIBot):
    def __init__(self, role='player'):
        super().__init__(role)

    def bet(self, current_state):
        player_score = calculate_score(self.hand) # 플레이어 점수 계산

        dealer_visible_card = current_state['dealer'][0]  # 딜러의 카드 확인

        if dealer_visible_card in ['J', 'Q', 'K']: # 딜러의 카드 점수 계산
            dealer_value = 10
        elif dealer_visible_card == 'A':
            dealer_value = 11
        else:
            dealer_value = int(dealer_visible_card)
        
        # 2장으로만 경쟁한다고 가정.
        #이길 가능성이 클 경우만 과감한 투자
        if player_score == 21:
            bet_money = 400  # 항상 이기는 경우
        elif player_score == 20: # 딜러가 21인 경우만 패배하므로 승률이 95.1%
          if dealer_value == 11: # 이 경우 68.5% 확률로 승리. 리스크가 큼
            bet_money = 80
          elif dealer_value == 10: # 92.1% 확률로 승리
            bet_money = 100
          else:
            bet_money = 400
        elif player_score == 19: # 20, 21인 경우 패배 가능. 리스크가 커짐.
          if dealer_value == 11:
            bet_money = 50
          elif dealer_value == 10:
            bet_money = 70
          else:  
            bet_money = 100
        else: # 최소금액 배팅
            bet_money = 20

        return bet_money

    def draw_decision(self, current_state):
        player_score = calculate_score(self.hand)

        # 딜러와 같은 판단. 중요하지 않음.
        if player_score >= 17:
            decision = False
        else:
            decision = True
        
        return decision

In [24]:
if __name__ == "__main__":

    player_win_cnt=0
    log_mode=True
    
    for game_num in range(10):
      if play_black_jack(log_mode):
        player_win_cnt+=1
    print(f"승률은 {player_win_cnt} 입니다")        

========================0번째 게임=================================
플레이어 버스트 딜러 승리 {'player': ['J', '3', '10'], 'dealer': ['3', '7', '6', 'J']} 플레이어 남은돈 480
딜러 승리 {'player': ['7', '5', '2'], 'dealer': ['9', '4', '7']} 플레이어 남은돈 460
딜러 버스트 플레이어 승리. {'player': ['A', '8'], 'dealer': ['2', 'J', 'Q']} 플레이어 남은돈 560
딜러 승리 {'player': ['6', '3', '7'], 'dealer': ['A', '9']} 플레이어 남은돈 540
플레이어 승리 {'player': ['A', 'J'], 'dealer': ['K', '2', '4', 'A']} 플레이어 남은돈 940
========================1번째 게임=================================
딜러 승리 {'player': ['9', 'K'], 'dealer': ['4', 'J', '2', '4']} 플레이어 남은돈 400
딜러 승리 {'player': ['4', 'A', 'K'], 'dealer': ['5', '6', '7']} 플레이어 남은돈 380
딜러 버스트 플레이어 승리. {'player': ['J', '8'], 'dealer': ['K', '2', '10']} 플레이어 남은돈 400
딜러 승리 {'player': ['8', '9'], 'dealer': ['2', '8', 'A']} 플레이어 남은돈 380
딜러 버스트 플레이어 승리. {'player': ['A', 'K'], 'dealer': ['Q', '6', 'Q']} 플레이어 남은돈 780
딜러 승리 {'player': ['7', 'K'], 'dealer': ['3', '6', '2', '9']} 플레이어 남은돈 760
플레이어 버스트 딜러 승리 {'player': ['6', 'K'

In [25]:
if __name__ == "__main__":

    player_win_cnt=0
    log_mode=False
    SAMPLE_NUM=500
    for game_num in range(SAMPLE_NUM):
      if play_black_jack(log_mode):
        player_win_cnt+=1
    print(f"승률은 {100*int(player_win_cnt)/SAMPLE_NUM}% 입니다")         
    

승률은 98.4% 입니다
